# Mineral Formula Calculation
**Jordan Lubbers, Chuck Lewis** 

*Oregon State University <br>
College of Earth, Ocean, and Atmopheric Sciences*<br>

## Motivation
Calculate a mineral's stoichiometry based on it's major element oxide measurements. This is outlined in [Deer et al., (1992)](https://www.semanticscholar.org/paper/An-Introduction-to-the-Rock-Forming-Minerals-Deer-Howie/16b1f203a4eaf1d3010c91868a03e8511a640238?p2df) and for a more detailed description, the reader is referred there. 

## Working
- Feldspars 
    - assumes all $Fe$ is $Fe^{3+}$ (e.g., Deer et al., (1992)).
- Olivine 
    - assumes all $Fe$ is $Fe^{2+}$. Probably safe as there is no $Al$ in olivine. 
- Pyroxene 
     - to distinguish between $Fe^{2+}$ and $Fe^{3+}$ we use [Droop 1987](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.627.9746&rep=rep1&type=pdf)


## To Do


## Example data format
- also list of supported elements
|sample|SiO2|TiO2|Al2O3|Cr2O3|FeO|BaO|SrO|MnO|CaO|Na2O|K2O|NiO|Total|
|------|----|----|-----|-----|---|---|---|---|---|----|---|---|-----|


## Workflow
We will code this by hand first and then show how it can be simply called using ```magmatrace```.
1. import our data of major element oxide analyses. 
2. normalize the data such that all totals are out of 100...this is optional later on in the actual function
3. calculate the molar proportion of oxides. This is done by:

$$ \large{X_{i_{mol}} = \frac{X_{i_{wt\%}}}{X_{i_{mol. wt}}}}$$

where $i$ is a given element. 

4. calculate the molar proportion of oxygen atoms for each cation. This is done by:

$$ \large{O_{i_{mol}} = X_{i_{mol}}\frac{n_{i_O}}{n_{i_X}}} $$

where $n_O$ is the number of oxygen atoms in the oxide's formula and $n_X$ is the number of cations in the oxide's formula. 

5. calculate the number of oxygens normalized to the ideal amount of oxygens in the chemical formula (e.g., 8 for feldspar). This is done by:

$$ \large{X_{i_{O_{norm}}} = \left[\frac{n_{O_{ideal}}}{\sum_{i=1}^{k} O_{i_{mol}}+...O_{k_{mol}}}\right]O_{i_{mol}}} $$



6. calculate the number of cations normalized to the ideal amount of oxygens in the chemical formula. This is done by:

$$\large{X_{i_{norm}} =  \frac{X_{i_{O_{norm}}}}{\left(\frac{{n_{i_O}}}{{n_{i_X}}}\right)}} $$ 
# Dependencies
- pandas
- numpy
- [mendeleev](https://github.com/lmmentel/mendeleev/)
- [re](https://docs.python.org/3/library/re.html)

In [1]:
import pandas as pd
import numpy as np
from mendeleev import element as el,  # not needed in the magmatrace version as molecular masses are hard coded
import re
import magmatrace_current as mt

## Import data
This will import the data as as standard pandas DataFrame. 

In [2]:
fsp_filepath = "mineral_recalc_test.xlsx"


# import the data
data = pd.read_excel(fsp_filepath)

# number of ideal oxygens
n_oxygens = 8

# this triggers the conditional loop below to calculate Fe2+ and Fe3+
mineral = "feldspar"


data.fillna(0, inplace=True)
# set the index to be the first column of the spreadsheet you import
data.set_index(list(data.columns)[0], inplace=True)
data.head()

,SiO2_wt,TiO2_wt,Al2O3_wt,Cr2O3_wt,FeO_t,BaO_wt,SrO_wt,MnO_wt,MgO_wt,CaO_wt,Na2O_wt,K2O_wt,Total_wt
spot,,,,,,,,,,,,,
LCTB_1_1,66.6815,0.0,18.9284,0.0,0.0845,0.2721,0.0,0.0,0.0,0.4877,4.9792,9.3485,100.7819
LCTB_1_2,66.1041,0.0,18.8633,0.0,0.1003,0.2422,0.0,0.0,0.0,0.3934,4.8596,9.1743,99.7372
LCTB_1_3,66.9309,0.0,18.9301,0.0,0.1002,0.2471,0.0,0.0,0.0,0.4139,5.0102,9.3585,100.9909
LCTB_1_4,66.2745,0.0,18.6919,0.0,0.1010,0.2647,0.0,0.0,0.0,0.4221,4.9888,9.2460,99.9890
LCTB_1_5,66.9421,0.0,18.7939,0.0,0.1426,0.2405,0.0,0.0,0.0,0.4720,4.8095,9.2955,100.6961


In [3]:
# Removes the 'total column' from the list
columns = list(data.columns)
elements = []
for column in columns:
    if "Total" in column:
        columns.remove(column)

# dropping anything after the space
for column in columns:
    elements.append(column.split(" ")[0])

# create new dataframe that is just the analyses without the total
oxides = data.loc[:, columns]
oxides.columns = elements
oxides.head()

,SiO2_wt,TiO2_wt,Al2O3_wt,Cr2O3_wt,FeO_t,BaO_wt,SrO_wt,MnO_wt,MgO_wt,CaO_wt,Na2O_wt,K2O_wt
spot,,,,,,,,,,,,
LCTB_1_1,66.6815,0.0,18.9284,0.0,0.0845,0.2721,0.0,0.0,0.0,0.4877,4.9792,9.3485
LCTB_1_2,66.1041,0.0,18.8633,0.0,0.1003,0.2422,0.0,0.0,0.0,0.3934,4.8596,9.1743
LCTB_1_3,66.9309,0.0,18.9301,0.0,0.1002,0.2471,0.0,0.0,0.0,0.4139,5.0102,9.3585
LCTB_1_4,66.2745,0.0,18.6919,0.0,0.1010,0.2647,0.0,0.0,0.0,0.4221,4.9888,9.2460
LCTB_1_5,66.9421,0.0,18.7939,0.0,0.1426,0.2405,0.0,0.0,0.0,0.4720,4.8095,9.2955


## Normalize the data
- This is optional and also not always the best way to do things!!!!!! (e.g., barometry and thermometry)

In [4]:
# normalize the wt%
oxides_normalized = 100 * (oxides.div(oxides.sum(axis="columns"), axis="rows"))
oxides_normalized.head()

,SiO2_wt,TiO2_wt,Al2O3_wt,Cr2O3_wt,FeO_t,BaO_wt,SrO_wt,MnO_wt,MgO_wt,CaO_wt,Na2O_wt,K2O_wt
spot,,,,,,,,,,,,
LCTB_1_1,66.164162,0.0,18.781547,0.0,0.083844,0.269989,0.0,0.0,0.0,0.483916,4.940570,9.275971
LCTB_1_2,66.278279,0.0,18.913003,0.0,0.100564,0.242838,0.0,0.0,0.0,0.394437,4.872405,9.198474
LCTB_1_3,66.274189,0.0,18.744362,0.0,0.099217,0.244676,0.0,0.0,0.0,0.409839,4.961041,9.266677
LCTB_1_4,66.281791,0.0,18.693956,0.0,0.101011,0.264729,0.0,0.0,0.0,0.422146,4.989349,9.247017
LCTB_1_5,66.479337,0.0,18.663980,0.0,0.141614,0.238837,0.0,0.0,0.0,0.468737,4.776253,9.231241


## calculate the mole cations for each oxide

In [5]:
mol_cations = np.zeros(oxides.shape)
oxides_normalized = oxides.copy()

for i, element in zip(range(len(elements)), elements):
    if "Si" in element:
        mol_cations[:, i] = oxides_normalized[element] / (
            el("Si").mass + (el("O").mass * 2)
        )
    elif "Ti" in element:
        mol_cations[:, i] = oxides_normalized[element] / (
            el("Ti").mass + (el("O").mass * 2)
        )
    elif "Al" in element:
        mol_cations[:, i] = (
            2 * oxides_normalized[element] / ((el("Al").mass * 2) + (el("O").mass * 3))
        )
    elif "Cr" in element:
        mol_cations[:, i] = (
            2 * oxides_normalized[element] / ((el("Cr").mass * 2) + (el("O").mass * 3))
        )
    elif "Fe" in element:
        mol_cations[:, i] = oxides_normalized[element] / (
            (el("Fe").mass) + (el("O").mass)
        )
    elif "Ba" in element:
        mol_cations[:, i] = oxides_normalized[element] / (
            (el("Ba").mass) + (el("O").mass)
        )
    elif "Sr" in element:
        mol_cations[:, i] = oxides_normalized[element] / (
            (el("Sr").mass) + (el("O").mass)
        )
    elif "Mn" in element:
        mol_cations[:, i] = oxides_normalized[element] / (
            (el("Mn").mass) + (el("O").mass)
        )
    elif "Mg" in element:
        mol_cations[:, i] = oxides_normalized[element] / (
            (el("Mg").mass) + (el("O").mass)
        )
    elif "Ca" in element:
        mol_cations[:, i] = oxides_normalized[element] / (
            (el("Ca").mass) + (el("O").mass)
        )
    elif "Na" in element:
        mol_cations[:, i] = (
            2 * oxides_normalized[element] / ((el("Na").mass * 2) + (el("O").mass))
        )
    elif "K" in element:
        mol_cations[:, i] = (
            2 * oxides_normalized[element] / ((el("K").mass * 2) + (el("O").mass))
        )
    elif "Ni" in element:
        mol_cations[:, i] = oxides_normalized[element] / (
            (el("Ni").mass) + (el("O").mass)
        )


mol_cations = pd.DataFrame(mol_cations, columns=elements)
mol_cations.head()

,SiO2_wt,TiO2_wt,Al2O3_wt,Cr2O3_wt,FeO_t,BaO_wt,SrO_wt,MnO_wt,MgO_wt,CaO_wt,Na2O_wt,K2O_wt
0,1.109823,0.0,0.371290,0.0,0.001176,0.001775,0.0,0.0,0.0,0.008697,0.160675,0.198491
1,1.100213,0.0,0.370013,0.0,0.001396,0.001580,0.0,0.0,0.0,0.007015,0.156816,0.194793
2,1.113974,0.0,0.371324,0.0,0.001395,0.001612,0.0,0.0,0.0,0.007381,0.161675,0.198704
3,1.103049,0.0,0.366651,0.0,0.001406,0.001726,0.0,0.0,0.0,0.007527,0.160985,0.196315
4,1.114160,0.0,0.368652,0.0,0.001985,0.001569,0.0,0.0,0.0,0.008417,0.155199,0.197366


## calculate the molar proportion of oxygen atoms for each oxide

In [6]:
mol_oxygens = np.zeros(mol_cations.shape)

for i, element in zip(range(len(elements)), elements):
    if "Si" in element:
        mol_oxygens[:, i] = mol_cations[element] * 2
    elif "Ti" in element:
        mol_oxygens[:, i] = mol_cations[element] * 2
    elif "Al" in element:
        mol_oxygens[:, i] = mol_cations[element] * (3 / 2)
    elif "Cr" in element:
        mol_oxygens[:, i] = mol_cations[element] * (3 / 2)
    elif "Fe" in element:
        mol_oxygens[:, i] = mol_cations[element] * 1
    elif "Ba" in element:
        mol_oxygens[:, i] = mol_cations[element] * 1
    elif "Sr" in element:
        mol_oxygens[:, i] = mol_cations[element] * 1
    elif "Mn" in element:
        mol_oxygens[:, i] = mol_cations[element] * 1
    elif "Mg" in element:
        mol_oxygens[:, i] = mol_cations[element] * 1
    elif "Ca" in element:
        mol_oxygens[:, i] = mol_cations[element] * 1
    elif "Na" in element:
        mol_oxygens[:, i] = mol_cations[element] * (1 / 2)
    elif "K" in element:
        mol_oxygens[:, i] = mol_cations[element] * (1 / 2)
    elif "Ni" in element:
        mol_oxygens[:, i] = mol_cations[element] * 1


mol_oxygens = pd.DataFrame(mol_oxygens, columns=elements)
mol_oxygens.head()

,SiO2_wt,TiO2_wt,Al2O3_wt,Cr2O3_wt,FeO_t,BaO_wt,SrO_wt,MnO_wt,MgO_wt,CaO_wt,Na2O_wt,K2O_wt
0,2.219646,0.0,0.556936,0.0,0.001176,0.001775,0.0,0.0,0.0,0.008697,0.080337,0.099246
1,2.200426,0.0,0.555020,0.0,0.001396,0.001580,0.0,0.0,0.0,0.007015,0.078408,0.097396
2,2.227948,0.0,0.556986,0.0,0.001395,0.001612,0.0,0.0,0.0,0.007381,0.080838,0.099352
3,2.206098,0.0,0.549977,0.0,0.001406,0.001726,0.0,0.0,0.0,0.007527,0.080492,0.098157
4,2.228321,0.0,0.552978,0.0,0.001985,0.001569,0.0,0.0,0.0,0.008417,0.077599,0.098683


## Calculate normalized cations and oxygens
- based on the ideal 8 oxygens for feldspars

In [7]:
# number of oxygens per cation, normalized to the ideal number of oxygens specified above

norm_oxygens = (mol_oxygens * n_oxygens).div(
    mol_oxygens.sum(axis="columns"), axis="rows"
)
norm_oxygens.head()

,SiO2_wt,TiO2_wt,Al2O3_wt,Cr2O3_wt,FeO_t,BaO_wt,SrO_wt,MnO_wt,MgO_wt,CaO_wt,Na2O_wt,K2O_wt
0,5.983251,0.0,1.501269,0.0,0.003170,0.004784,0.0,0.0,0.0,0.023443,0.216557,0.267525
1,5.985027,0.0,1.509622,0.0,0.003797,0.004297,0.0,0.0,0.0,0.019081,0.213264,0.264912
2,5.990093,0.0,1.497520,0.0,0.003750,0.004333,0.0,0.0,0.0,0.019844,0.217341,0.267119
3,5.992014,0.0,1.493800,0.0,0.003818,0.004689,0.0,0.0,0.0,0.020445,0.218626,0.266607
4,6.003117,0.0,1.489728,0.0,0.005347,0.004226,0.0,0.0,0.0,0.022675,0.209054,0.265853


In [8]:
# calculate the mole cations of each oxide normalized to the number of ideal oxygens
norm_cations = np.zeros(norm_oxygens.shape)

for i, element in zip(range(len(elements)), elements):
    if "Si" in element:
        norm_cations[:, i] = norm_oxygens[element] / 2
    elif "Ti" in element:
        norm_cations[:, i] = norm_oxygens[element] / 2
    elif "Al" in element:
        norm_cations[:, i] = norm_oxygens[element] / (3 / 2)
    elif "Cr" in element:
        norm_cations[:, i] = norm_oxygens[element] / (3 / 2)
    elif "Fe" in element:
        norm_cations[:, i] = norm_oxygens[element]
    elif "Ba" in element:
        norm_cations[:, i] = norm_oxygens[element]
    elif "Sr" in element:
        norm_cations[:, i] = norm_oxygens[element]
    elif "Mn" in element:
        norm_cations[:, i] = norm_oxygens[element]
    elif "Mg" in element:
        norm_cations[:, i] = norm_oxygens[element]
    elif "Ca" in element:
        norm_cations[:, i] = norm_oxygens[element]
    elif "Na" in element:
        norm_cations[:, i] = norm_oxygens[element] / (1 / 2)
    elif "K" in element:
        norm_cations[:, i] = norm_oxygens[element] / (1 / 2)
    elif "Ni" in element:
        norm_cations[:, i] = norm_oxygens[element]

cations = []
# Get the cations by taking the first two characters
[cations.append(element[:2]) for element in elements]

# since some elements are only one letter (e.g., K) this
# strips the number from it

r = re.compile("([a-zA-Z]+)([0-9]+)")
for i in range(len(cations)):

    m = r.match(cations[i])
    if m != None:
        cations[i] = m.group(1)

norm_cations = pd.DataFrame(norm_cations, columns=cations)
norm_cations["Total_cations"] = norm_cations.sum(axis="columns")
norm_cations[data.index.name] = data.index.tolist()


if mineral == "pyroxene":
    # ideal cations
    T = 4

    # calculated cations based on oxide measurements
    S = norm_cations["Total_cations"]

    # step 2 and 3 from Droop 1987
    norm_cations.loc[norm_cations["Total_cations"] > T, "Fe_3"] = (
        2 * n_oxygens * (1 - (T / norm_cations["Total_cations"]))
    )
    norm_cations.loc[norm_cations["Total_cations"] <= T, "Fe_3"] = 0

    # step 4 from Droop 1987
    norm_cations.set_index(data.index.name, inplace=True)

    ts = T / norm_cations["Total_cations"].to_numpy()
    norm_cations = norm_cations * ts[:, np.newaxis]

    norm_cations["Fe_2"] = norm_cations["Fe"] - norm_cations["Fe_3"]

else:

    norm_cations.set_index(data.index.name, inplace=True)
norm_cations.head()

,Si,Ti,Al,Cr,Fe,Ba,Sr,Mn,Mg,Ca,Na,K,Total_cations
spot,,,,,,,,,,,,,
LCTB_1_1,2.991626,0.0,1.000846,0.0,0.003170,0.004784,0.0,0.0,0.0,0.023443,0.433114,0.535051,4.992033
LCTB_1_2,2.992513,0.0,1.006414,0.0,0.003797,0.004297,0.0,0.0,0.0,0.019081,0.426529,0.529824,4.982456
LCTB_1_3,2.995047,0.0,0.998346,0.0,0.003750,0.004333,0.0,0.0,0.0,0.019844,0.434683,0.534237,4.990240
LCTB_1_4,2.996007,0.0,0.995867,0.0,0.003818,0.004689,0.0,0.0,0.0,0.020445,0.437253,0.533214,4.991293
LCTB_1_5,3.001558,0.0,0.993152,0.0,0.005347,0.004226,0.0,0.0,0.0,0.022675,0.418107,0.531706,4.976772


## Printing the mineral formula

In [9]:
def get_min_formula(df, row):
    """function to extract a row from a dataframe
    Inputs:
    df = pandas dataframe object
    row = index value you want to index by. Must be 
    a string index
    
    
    """
    formula = df.loc[row, :]
    return formula

In [10]:
formula = get_min_formula(norm_cations, "LCTB_1_1")
formula

Si               2.991626
Ti               0.000000
Al               1.000846
Cr               0.000000
Fe               0.003170
Ba               0.004784
Sr               0.000000
Mn               0.000000
Mg               0.000000
Ca               0.023443
Na               0.433114
K                0.535051
Total_cations    4.992033
Name: LCTB_1_1, dtype: float64

## Function to save a dataframe as an Excel spreadsheet

In [11]:
def save_df(df, outpath, outname):
    """function to save a pandas dataframe as an Excel spreadsheet
    
    Inputs:
    df: pandas dataframe
    outpath: filepath of where to save the output spreadsheet
    outname: name of the file without .xlsx
    
    Returns:
    xlsx spreadsheet saved to the specified directory
    """

    # This is creating a writer object to save your dataframe as an excel spreadsheet
    #'strings_to_numbers': True avoids the Excel error "numbers stored as text"
    writer = pd.ExcelWriter(
        "{}/{}.xlsx".format(outpath, outname),
        engine="xlsxwriter",
        options={"strings_to_numbers": True},
    )

    # write the dataframe to a an XlsxWriter Excel object
    # index = False makes it so the index [0,1,2,3,...] is not included in spreadsheet
    df.to_excel(writer, sheet_name="Sheet1", index=True)

    # Close the pandas Excel writer and output the Excel file to the directory specified
    writer.save()

    print(
        "Your file has been saved as the following: {}/{}.xslx".format(outpath, outname)
    )

## Using ```magmatrace```

Here we:
1. bring in the same spreadsheet as above
2. call the ```mineral_formula_recalc``` function from ```magmatrace```

```
mineral_formula_calc(df, n_oxygens, mineral, normalized, index)
    mineral_formula_calc is a function that calculates the stoichiometry for a mineral based on a set of major
    element oxide analyses as described by Deer et al., 1966 Appendix 1
    
    Inputs:
    df : pandas dataframe object of major element analyses. Column headers must have the the element somewhere in the name
    
    ** if a column containing 'Total' in the name exists, it will be removed so that only the individual analyses are 
    present
    ** your dataframe should have a column that pertains to sample, analysis number, etc. This will be set as the index
    of the dataframe so that chemical formulas can be accessed easily upon calculation
    
    EXAMPLE OF INPUT DATAFRAME:
    |sample|SiO2|TiO2|Al2O3|Cr2O3|FeO|BaO|SrO|MnO|CaO|Na2O|K2O|NiO|Total| <---- currently supported elements
    
    
    n_oxygens : number of ideal oxygens in the chemical formula (e.g., for feldspars this would be 8)
    
    mineral : 'feldspar','olivine','pyroxene'
    if 'pyroxene' is chosen, the function will calculate the proportions of Fe2+ and Fe3+ based off stoichiometry and charge
    balance as described by Droop 1987. If 'feldspar', all Fe is assumed to be Fe3+. If 'olivine', all Fe is assumed to be 2+
    
    normalized: boolean 
    if True, will normalize your geochemical analyses. If false, mineral formulas will be calculated using 
    raw geochemical data
    
    index: string
    column denoting which column to be used as the index for the dataframe. Suggested that this is a column that 
    denotes sample name or spot name or something similar
    
    
    Returns:
    norm_cations: pandas dataframe object that contains the calculated number of cations in the chemical formula
    normalized to the amount of ideal oxygens specified by 'n_oxygens'.
```
3. call the ```get_min_formula``` function
4. call the ```save_df``` function

Now the entire workflow from above can be fit in the following few lines of code

In [12]:
# import data
data = pd.read_excel(fsp_filepath)
data.head()

,spot,SiO2_wt,TiO2_wt,Al2O3_wt,Cr2O3_wt,FeO_t,BaO_wt,SrO_wt,MnO_wt,MgO_wt,CaO_wt,Na2O_wt,K2O_wt,Total_wt
0,LCTB_1_1,66.6815,NaN,18.9284,NaN,0.0845,0.2721,NaN,NaN,NaN,0.4877,4.9792,9.3485,100.7819
1,LCTB_1_2,66.1041,NaN,18.8633,NaN,0.1003,0.2422,NaN,NaN,NaN,0.3934,4.8596,9.1743,99.7372
2,LCTB_1_3,66.9309,NaN,18.9301,NaN,0.1002,0.2471,NaN,NaN,NaN,0.4139,5.0102,9.3585,100.9909
3,LCTB_1_4,66.2745,NaN,18.6919,NaN,0.1010,0.2647,NaN,NaN,NaN,0.4221,4.9888,9.2460,99.9890
4,LCTB_1_5,66.9421,NaN,18.7939,NaN,0.1426,0.2405,NaN,NaN,NaN,0.4720,4.8095,9.2955,100.6961


In [13]:
# call the function with all the requisite keyword arguments
mineral_formulas = mt.mineral_formula_calc(
    data, 
    n_oxygens=8, 
    mineral="feldspar",
    normalized=False,
    index="spot"
)
mineral_formulas.head()

,Si,Ti,Al,Cr,Fe,Ba,Sr,Mn,Mg,Ca,Na,K,Total_cations
spot,,,,,,,,,,,,,
LCTB_1_1,2.991571,0.0,1.000945,0.0,0.003170,0.004784,0.0,0.0,0.0,0.023444,0.433006,0.535078,4.991999
LCTB_1_2,2.992458,0.0,1.006514,0.0,0.003797,0.004297,0.0,0.0,0.0,0.019082,0.426423,0.529851,4.982422
LCTB_1_3,2.994992,0.0,0.998445,0.0,0.003750,0.004333,0.0,0.0,0.0,0.019845,0.434575,0.534265,4.990205
LCTB_1_4,2.995953,0.0,0.995966,0.0,0.003818,0.004689,0.0,0.0,0.0,0.020446,0.437145,0.533241,4.991257
LCTB_1_5,3.001503,0.0,0.993250,0.0,0.005347,0.004226,0.0,0.0,0.0,0.022676,0.418003,0.531733,4.976740


In [14]:
formula = get_min_formula(mineral_formulas, "LCTB_1_1")
formula

Si               2.991571
Ti               0.000000
Al               1.000945
Cr               0.000000
Fe               0.003170
Ba               0.004784
Sr               0.000000
Mn               0.000000
Mg               0.000000
Ca               0.023444
Na               0.433006
K                0.535078
Total_cations    4.991999
Name: LCTB_1_1, dtype: float64

In [16]:
save_df(mineral_formulas, "/Users/jordanlubbers/Desktop", "mineral_formula_output_test")

Your file has been saved as the following: /Users/jordanlubbers/Desktop/mineral_formula_output_test.xslx


/Users/jordanlubbers/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
